### Imports and installs

In [2]:
!pip install esm
!pip install omegaconf
!pip install torch
!pip install huggingface-hub

INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.20.0-cp311-cp311-win_amd64.whl.metadata (6.2 kB)
Using cached torchvision-0.20.0-cp311-cp311-win_amd64.whl (1.6 MB)
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2
    Uninstalling torchvision-0.15.2:
      Successfully uninstalled torchvision-0.15.2


In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn

from model import DNADecoder

from omegaconf import OmegaConf
from torch.nn import TransformerDecoderLayer, TransformerDecoder
from transformers import AutoModel, AutoTokenizer

ImportError: attempted relative import with no known parent package

### Set up models

In [4]:
config = OmegaConf.load("config.yaml")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
choformer = DNADecoder(config).to(device).eval()
choformer.load_state_dict(torch.load("best_model.pth", map_location=device))

esm_model = AutoModel.from_pretrained("facebook/esm2_t6_8M_UR50D").to(device)
esm_tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

/var/folders/ff/pbhg09jd53nb6lhsv863ym9c0000gn/T/ipykernel_75791/2697218582.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  choformer.load_state_dict(torch.load("best_mo

In [9]:
# import protein sequences
import pandas as pd
proteins = "/Users/rishabjain/Desktop/Research/choformer/inference/cho_low_exp_prot.csv"

df = pd.read_csv(proteins)

# convert column protein to list of sequences
sequences = df["protein"].tolist()

### Inference

In [12]:
protein_sequences = sequences[:10]
longest_protein_length = max([len(sequence) for sequence in protein_sequences])

protein_tokens = esm_tokenizer(
    protein_sequences,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=longest_protein_length
).to(device)


with torch.no_grad():
    protein_embeddings = esm_model(**protein_tokens).last_hidden_state.squeeze(0)

In [13]:
outputs = choformer.generate(protein_embeddings)
print("GENERATED DNA SEQUENCES:", outputs['generated_sequences'])

GENERATED DNA SEQUENCES: ['ATGTCCTGGACCTCTGCCGACCCCGGGCAGAGCTCCCGCCCCCGCTGCCGCTTCCTGGAGGCCTCCCGCCTGGAGCGCCCCGTGCAGCTGGCCTGCTCCCGCCTGGAGCGCCGCTTCCGCGTGGAGCCCCGCGTCACCTTCCCGCCCGCCGAGCTCTGCTGCCGCCAGCGCCGCCGCCGCCGCGCGGCCTGCGGCCGCCGCGGCCTGGGCTGCGGCGCCCAGCTCAGCGCCAGCCCCGGCACCGGCGGCGTCTTCGGGGCCCCGAGCCCCGGGCGCCGCGCCGGCGCCGGGCCCGGCGGCCGCGCGGCCGGCGCCTGGCCGCCGTGCGCCGCGCTGCTGGTGGGCCGCGGCGAGGACGGCCCCGCCCGGGCCACCCTGGGCGCCGGCACCTCCCGCCCCCGGGACCCCGGGCCCTGGACCGCCGCCCGCCCCTGGCACGTGCGCCGCAGCCTGGAGGCCGCCTGGCCGCGCCTGGCCGGCCTGGCCCCGGCGGAGGCCGGCTGCGCCCGCCGCCGCGTCTGCCCGCGCGTCTCCCCCTCCCCCTTCTGCGGCAGCAACGCCTGCCTGCCCGGCGGCGCCCGGCAGCAGGTGCCCGGGGTCTGCGCCGCCCGGCAGCTCTCCTCCTCCAGCCAGCGCCGCAGCGAGGCCCTGGCCGGGGCCCCCCTGGACAATGCCCCCAAGGAGTACCCCCCCAAGATCCAGCAGCTGGTGCAGGACATTGCCAGCCTGACCCTGCTGGAGATCAGTGACCTGAATGAGCTGCTGAAGAAGACCCTGAAGATCCAGGATGTGGGCCTGATGCCCATGGGGGGCATGATGCCTGGGGCTGTGCCTGCCGCCGCCGCCGCCGCCCCCGAGGTGGCTGAGGGGGAGGACATCCCCAAGCAGAAGGAGCGCACCCACTTTACAGTGCGCCTGACAGAGGCCAAGCCTGTGGACAAGGTGAAGCTGATCAAGGAGATCAAGAATTATG